In [1]:
import pandas as pd

pd.set_option('display.max_columns', 100)

subset_pth = "~/ds7900_spring23_team2/data_process/data/subsets/"

def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'
    
def within_k_days(DAYS_SINCE_LAST_x, k):
    res = 0
    if DAYS_SINCE_LAST_x <= k:
        res = 1
    return res

In [ ]:
df = pd.read_csv(subset_pth + "final_validation_df_v2.csv", dtype = {"HASH_NBR": "object"})
test_df = pd.read_csv(subset_pth + "email_hist_test.csv", dtype = {"HASH_NBR": "object"})

test_df = test_df[test_df["HASH_NBR"] != "HASH_NBR"]

df.drop_duplicates(subset = ["HASH_NBR"],
                   keep = "last",
                   inplace = True,
                   ignore_index = True                              
                  )

df.loc[df["CLICK"] == 1, "total_clicks"] = df.loc[df["CLICK"] == 1, "total_clicks"] - 1
df.loc[df["total_clicks"] < 0, "total_clicks"] = 0
df["CLICK_THROUGH_RATE"] = (df["total_clicks"] / df["total_emails"]).fillna(0)

df[f"CLICK_WTHN_{60}_DAYS"] = [within_k_days(x, 60) for x in df["DAYS_SINCE_LAST_CLICK"]]

model_set_2 = ['CLICK_THROUGH_RATE', 'DAYS_SINCE_LAST_CLICK', 'CLICK_WTHN_60_DAYS', 'GNDR_1', 'GNDR_2', 'EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_6', 'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5', 'INCOME_1', 'INCOME_2', 'INCOME_3', 'INCOME_4', 'INCOME_5']

df = pd.merge(left = test_df,
              right = df.loc[:, ["HASH_NBR"] + model_set_2],
              left_on = "HASH_NBR",
              right_on = "HASH_NBR",
              how = "left",
              suffixes = ["", "_prior"]
             )

df["DAYS_SINCE_LAST_CLICK"].fillna(df["DAYS_SINCE_LAST_CLICK"].max(), inplace = True)

df["DAYS_SINCE_LAST_CLICK"] = df["DAYS_SINCE_LAST_CLICK"].astype('int64')

for col in ['CLICK_THROUGH_RATE', 'CLICK_WTHN_60_DAYS', 
           'GNDR_1', 'GNDR_2',
           'EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_6',
           'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5',
           'INCOME_1', 'INCOME_2', 'INCOME_3', 'INCOME_4', 'INCOME_5'
          ]:
    df.loc[:, col].fillna(0, inplace = True)
    
df["SEND_DT"] = pd.to_datetime(df["SEND_DT"])

df['quarter'] = df['SEND_DT'].dt.quarter
df['week_of_year'] = df['SEND_DT'].dt.strftime("%W")
df['day_of_week'] = df['SEND_DT'].dt.day_name()
df['day_of_month'] = df['SEND_DT'].dt.day
df['day_of_year'] = df['SEND_DT'].dt.dayofyear
df['month'] = df['SEND_DT'].dt.month
df['is_end_month'] = df['SEND_DT'].dt.is_month_end.astype('int8')
df['is_start_month'] = df['SEND_DT'].dt.is_month_start.astype('int8')
df['is_end_quarter'] = df['SEND_DT'].dt.is_quarter_end.astype('int8')
df['is_start_quarter'] = df['SEND_DT'].dt.is_quarter_start.astype('int8')

df['season'] = df['month'].apply(get_season)
df['month'] = df['SEND_DT'].dt.month_name()

df["CC_2"] = 0   
df["CC_3"] = 0   
df["CC_4"] = 0   
df["TIER_2"] = 0   
df["TIER_3"] = 0   
df["TIER_4"] = 0   
df["quarter_1"] = 0   
df["quarter_3"] = 0   
df["quarter_4"] = 0
df["January"] = 0   
df["February"] = 0   
df["April"] = 0   
df["May"] = 0   
df["June"] = 0     
df["July"] = 0   
df["August"] = 0   
df["September"] = 0   
df["October"] = 0   
df["November"] = 0   
df["December"] = 0    
df["Monday"] = 0   
df["Tuesday"] = 0   
df["Wednesday"] = 0   
df["Thursday"] = 0   
df["Friday"] = 0   
df["Saturday"] = 0     
df["Fall"] = 0   
df["Summer"] = 0   
df["Winter"] = 0    

df.loc[df["CAMPAIGN_NM"] == 1, "CC_2"] = 1
df.loc[df["CAMPAIGN_NM"] == 2, "CC_3"] = 1
df.loc[df["CAMPAIGN_NM"] == 3, "CC_4"] = 1
df.loc[df["MBR_TIER"] == 1, "TIER_2"] = 1
df.loc[df["MBR_TIER"] == 2, "TIER_3"] = 1
df.loc[df["MBR_TIER"] == 3, "TIER_4"] = 1
df.loc[df["quarter"] == 1, "quarter" + "_" + "1"] = 1
df.loc[df["quarter"] == 3, "quarter" + "_" + "3"] = 1
df.loc[df["quarter"] == 4, "quarter" + "_" + "4"] = 1
df.loc[df["month"] == 0, "January"] = 1
df.loc[df["month"] == 1, "February"] = 1
df.loc[df["month"] == 3, "April"] = 1
df.loc[df["month"] == 4, "May"] = 1
df.loc[df["month"] == 5, "June"] = 1
df.loc[df["month"] == 0, "July"] = 1
df.loc[df["month"] == 1, "August"] = 1
df.loc[df["month"] == 2, "September"] = 1
df.loc[df["month"] == 3, "October"] = 1
df.loc[df["month"] == 4, "November"] = 1
df.loc[df["month"] == 5, "December"] = 1
df.loc[df["day_of_week"] == 0, "Monday"] = 1
df.loc[df["day_of_week"] == 1, "Tuesday"] = 1
df.loc[df["day_of_week"] == 2, "Wednesday"] = 1
df.loc[df["day_of_week"] == 3, "Thursday"] = 1
df.loc[df["day_of_week"] == 4, "Friday"] = 1
df.loc[df["day_of_week"] == 5, "Saturday"] = 1
df.loc[df["season"] == 'Fall', "Fall"] = 1
df.loc[df["season"] == 'Summer', "Summer"] = 1
df.loc[df["season"] == 'Winter', "Winter"] = 1

df["MBR_PRGM_ACTV"] = [1 if x == "LFC_3" else 0 for x in df["MBR_PRGM_ACTV"]]

col_list = ['CLICK_WTHN_60_DAYS', 'January', 'February', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Fall', 'Summer', 'Winter', 'week_of_year', 'day_of_month', 'day_of_year', 'is_end_month', 'is_start_month', 'is_end_quarter', 'is_start_quarter', 'MBR_PRGM_ACTV', 'CC_2', 'CC_3', 'CC_4', 'TIER_2', 'TIER_3', 'TIER_4', 'quarter_1', 'quarter_3', 'quarter_4', 'GNDR_1', 'GNDR_2', 'EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_6', 'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5', 'INCOME_1', 'INCOME_2', 'INCOME_3', 'INCOME_4', 'INCOME_5']

df.loc[:, col_list] = df.loc[:, col_list].astype('int8')

model_set_2 = ['CLICK_THROUGH_RATE', 'DAYS_SINCE_LAST_CLICK', 'CLICK_WTHN_60_DAYS', 'January', 'February', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Fall', 'Summer', 'Winter', 'week_of_year', 'day_of_month', 'day_of_year', 'is_end_month', 'is_start_month', 'is_end_quarter', 'is_start_quarter', 'MBR_PRGM_ACTV', 'CC_2', 'CC_3', 'CC_4', 'TIER_2', 'TIER_3', 'TIER_4', 'quarter_1', 'quarter_3', 'quarter_4', 'GNDR_1', 'GNDR_2', 'EN_1', 'EN_2', 'EN_3', 'EN_4', 'EN_6', 'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5', 'INCOME_1', 'INCOME_2', 'INCOME_3', 'INCOME_4', 'INCOME_5']

df[model_set_2].head(3)

In [ ]:
df[model_set_2].shape

In [ ]:

X = df[model_set_2].values

In [ ]:
import pickle

model_pth = "N:/Classes/2023_1SPRING/Data 7900 - Prof. Franks - Applied Analysis Project/Code/models/"

model_name_list = ["XGBoost", "NaiveBayes", "LogisticRegression", "RandomForest", "DecisionTree"]

for model_name in model_name_list:
    with open(model_pth + model_name + ".pkl", 'rb') as file:
        model = pickle.load(file)
        df[model_name + "_pred"] = model.predict(X)
        df[model_name + "_prob"] = model.predict_proba(X)[:, 1]

        print(df.loc[:, ["HASH_NBR", model_name + "_pred"]].groupby(model_name + "_pred").count() / 8888824)
        

In [ ]:
df.loc[:, ["HASH_NBR", "CAMPAIGN_NBR", "SEND_DT"] + temp_list].to_csv("test_results.csv", index = True)

In [ ]:
df.loc[:, ["HASH_NBR", "CAMPAIGN_NBR", "SEND_DT", "XGBoost_pred", "XGBoost_prob", "NaiveBayes_pred", "NaiveBayes_prob"]].to_csv("best_test_results.csv", index = True)